In [6]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars org.apache.spark:spark-streaming-kafka-0-8-assembly_2.11-2.3.2.jar pyspark-shell'


In [8]:
import argparse
import logging
import json
import mysql
from kafka import KafkaConsumer, KafkaProducer
from mysql.connector import errorcode

import logging
from cassandra.cqlengine import columns
from cassandra.cqlengine.models import Model
from cassandra.cluster import Cluster, BatchStatement
from cassandra.query import SimpleStatement

from kafka.client import KafkaClient
from kafka.consumer import SimpleConsumer
from cassandra.cluster import Cluster
from cassandra.policies import DCAwareRoundRobinPolicy
import datetime
import time
import sys
from datetime import datetime
from pdb import set_trace



import findspark
findspark.init('/usr/local/spark/spark-2.3.2-bin-hadoop2.7')

import pyspark
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as f
from pyspark.sql.types import DateType, StringType
from pyspark.sql import *
spark = SparkSession \
    .builder \
    .appName("Poolminers") \
    .getOrCreate()
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils


class PythonCassandra:
    def __init__(self):
        self.cluster = None
        self.session = None
        self.keyspace = None
        self.log = None

    def __del__(self):
        self.cluster.shutdown()

    def createsession(self):
        self.cluster = Cluster(['localhost'])
        self.session = self.cluster.connect(self.keyspace)

    def getsession(self):
        return self.session

    # How about Adding some log info to see what went wrong
    def setlogger(self):
        log = logging.getLogger()
        log.setLevel('INFO')
        handler = logging.StreamHandler()
        handler.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] %(name)s: %(message)s"))
        log.addHandler(handler)
        self.log = log

    # Create Keyspace based on Given Name
    def createkeyspace(self, keyspace):
        """
        :param keyspace:  The Name of Keyspace to be created
        :return:
        """
        # Before we create new lets check if exiting keyspace; we will drop that and create new
        rows = self.session.execute("SELECT keyspace_name FROM system_schema.keyspaces")
        keyspace_exists = False
        if keyspace in [row[0] for row in rows]:
            keyspace_exists = True
        if keyspace_exists is False:
            # self.log.info("dropping existing keyspace...")
            # self.session.execute("DROP KEYSPACE " + keyspace)
            self.log.info("creating keyspace...")
            self.session.execute("""
                CREATE KEYSPACE %s
                WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': '1' }
                """ % keyspace)
            self.log.info("setting keyspace...")
        self.session.set_keyspace(keyspace)

    def create_table_block(self):
        c_sql = """
                CREATE TABLE IF NOT EXISTS block (block_number bigint, block_hash varchar,
                                              miner_address varchar, parent_hash varchar, receipt_tx_root varchar,
                                              state_root varchar, tx_trie_root varchar, extra_data varchar, 
                                              nonce varchar, bloom varchar, solution varchar, difficulty varchar, 
                                              total_difficulty varchar, nrg_consumed bigint, nrg_limit bigint,
                                              block_size bigint, block_timestamp bigint, num_transactions bigint,
                                              block_time bigint, nrg_reward varchar, transaction_id bigint,
                                              transaction_list varchar,
                                              PRIMARY KEY (block_timestamp,block_number)
                                              );
                 """
        self.session.execute(c_sql)
        self.log.info("Block Table Created !!!")

    def create_table_transaction(self):
        c_sql = """
                CREATE TABLE IF NOT EXISTS transaction (id bigint,
                                              transaction_hash varchar, block_hash varchar, block_number bigint,
                                              transaction_index bigint, from_addr varchar, to_addr varchar, 
                                              nrg_consumed bigint, nrg_price bigint, transaction_timestamp bigint,
                                              block_timestamp bigint, tx_value varchar, transaction_log varchar,
                                              tx_data varchar, nonce varchar, tx_error varchar, contract_addr varchar,
                                              PRIMARY KEY (block_timestamp,block_number,transaction_timestamp)
                                              );
                 """
        self.session.execute(c_sql)
        self.log.info("Transaction Table Created !!!")

    def insert_data_transaction(self, message):
        insert_sql = self.session.prepare(
            """ INSERT INTO transaction(
            id,transaction_hash, block_hash, block_number,
            transaction_index, from_addr, to_addr, 
            nrg_consumed, nrg_price, transaction_timestamp,
            block_timestamp, tx_value, transaction_log,
            tx_data, nonce, tx_error, contract_addr)
            VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
            """
        )
        batch = BatchStatement()
        # batch.add(insert_sql, (1, 'LyubovK'))
        batch.add(insert_sql, message)
        self.session.execute(batch)
        self.log.info('Batch Insert Completed')

    def insert_data_block(self, message):
        insert_sql = self.session.prepare("""
                                            INSERT INTO block(block_number, block_hash, miner_address, 
                                            parent_hash, receipt_tx_root,
                                            state_root, tx_trie_root, extra_data, 
                                            nonce, bloom, solution, difficulty, 
                                            total_difficulty, nrg_consumed, nrg_limit,
                                            block_size, block_timestamp, num_transactions,
                                            block_time, nrg_reward, transaction_id, transaction_list) 
                                            VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
                                            """)
        batch = BatchStatement()
        # batch.add(insert_sql, (1, 'LyubovK'))
        batch.add(insert_sql, message)

        self.session.execute(batch)
        self.log.info('Block Batch Insert Completed')

    def select_data(self, table):
        rows = self.session.execute('select * from ' + table)
        for row in rows:
            print(row)

    def update_data(self):
        pass

    def delete_data(self):
        pass

class KafkaConnectPyspark:
    def __init__(self):
        os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars org.apache.spark:spark-streaming-kafka-0-8-assembly_2.11-2.3.2.jar pyspark-shell'
        self.sc = SparkContext.getOrCreate(spark)
        self.ssc = StreamingContext(self.sc,60) #
        self.kafkaStream=None

    def connect(self):
        #broker, topic = sys.argv[1:]
        broker='localhost:2181'
        topic='aionv4server.aionv4.block'
        self.kafkaStream = KafkaUtils.createStream(self.ssc,broker,
                                                   'block-streaming-consumer', {topic: 1})

        rows = self.kafkaStream.map(lambda x: x[1])
        rows.pprint()
        self.ssc.start()
        self.ssc.awaitTermination()

Exception: Java gateway process exited before sending its port number

In [4]:
kcp = KafkaConnectPyspark()
kcp.connect()
kcp.kafkaStream.pprint()

NameError: name 'KafkaConnectPyspark' is not defined

In [3]:
print(os.environ)

environ({'CONDA_PYTHON_EXE': '/home/andre/anaconda3/bin/python', 'GNOME_SHELL_SESSION_MODE': 'ubuntu', 'LC_NAME': 'es_PR.UTF-8', 'XDG_SESSION_TYPE': 'x11', 'GPG_AGENT_INFO': '/run/user/1000/gnupg/S.gpg-agent:0:1', 'QT_IM_MODULE': 'ibus', 'GDMSESSION': 'ubuntu', 'GTK2_MODULES': 'overlay-scrollbar', 'PROJ_LIB': '/home/andre/anaconda3/envs/bokeh_aion_analytics/share/proj', 'XMODIFIERS': '@im=ibus', 'SDKMAN_CANDIDATES_DIR': '/home/andre/.sdkman/candidates', 'LESSCLOSE': '/usr/bin/lesspipe %s %s', '_': '/home/andre/anaconda3/envs/bokeh_aion_analytics/bin/jupyter', 'QT_ACCESSIBILITY': '1', 'GRADLE_HOME': '/home/andre/.sdkman/candidates/gradle/current', 'KAFKA_HOME': '/home/kafka/kafka', 'SPARK_HOME': '/usr/local/spark/spark-2.3.2-bin-hadoop2.7', 'XDG_MENU_PREFIX': 'gnome-', '_CONDA_SET_PROJ_LIB': 'PROJ_LIB', 'XDG_CURRENT_DESKTOP': 'ubuntu:GNOME', 'TEXTDOMAIN': 'im-config', 'XDG_VTNR': '2', 'PAGER': 'cat', 'DISPLAY': ':0', 'HOME': '/home/andre', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/use

In [5]:
kcp.sc.stop()

NameError: name 'kcp' is not defined